In [1]:
!pip install transformers #NLP (Doğa Dil İşleme)'yi kullanmak için transformers kütüphanesi yüklenmiştir.

     |████████████████████████████████| 2.1MB 15.7MB/s 
     |████████████████████████████████| 870kB 50.7MB/s 
     |████████████████████████████████| 3.3MB 37.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=f0ce52b76a70bcf125615849d079899386b60caaf26dc6b2ca6d23f41e52bb92
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [2]:
import transformers
import pandas as pd

In [3]:
df=pd.read_csv('ses_kayitlari_metin.csv' , sep='delimiter',  engine='python', encoding='cp1254') #Veri seti yüklenmiştir.

In [4]:
#Burada BERT'in Türkçe Duygu Tespiti yapabilmesi için oluşturulan model yüklenir.
#Model yapısı içinde iki duygu etiketi bulunmaktadır.Bunlar; pozitif ve negatiftir.

from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
model = AutoModelForSequenceClassification.from_pretrained("savasy/bert-base-turkish-sentiment-cased")

#BERT yapısı içinde bulunan tokenizer nesnesi kullanılır ve verileri tokenlerine ayırma işlemi gerçekleştirilir.
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-sentiment-cased")

#pipeline nesnesi verinin peşpeşe seri biçimde tanımlanmış birtakım işlemlerden geçmesine olanak sağlar. 
#Önce tokenleştirme işlemi gerçekleştiriliyor ve veriler modele verilerek duygu tespitinin gerçekleştirilmesi sağlanıyor.
sa= pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)

analysisResultList = []

for str in df.values:
  analysisRowItem = []
  analysisResult= sa(str[0])
  
  analysisResult[0]['sentence'] = str.item()

  print(analysisResult)

#Veri setindeki verilerin duygu tespiti gerçekleştirildikten sonra etiket, skor ve cümle şeklinde sütunlara ayrılmıştır. 
  analysisRowItem.append(analysisResult[0]['label'])
  analysisRowItem.append(analysisResult[0]['score'])
  analysisRowItem.append(analysisResult[0]['sentence'])
  
  analysisResultList.append(analysisRowItem)

#Çıkan sonuçlar; Sentiment, Score ve Sentence şeklinde sütunlara ayrılmış ve BertTurkishSA excel dosyasına aktarılmıştır.
excelData = pd.DataFrame(analysisResultList, columns=['Sentiment','Score','Sentence']) 

excelData.to_excel('BertTurkishSA.xlsx')


Görüntülenen çıkış son 5000 satıra kısaltıldı.
[{'label': 'negative', 'score': 0.9943585395812988, 'sentence': 'Seydiu şimdi iki mevkiyi de kaybetti.'}]
[{'label': 'negative', 'score': 0.9446415901184082, 'sentence': 'Fakat öte yandan, hayatta gelişim sağlayabilmek için kendimi ne kadar çok strese soktuğumu farkediyorum.'}]
[{'label': 'positive', 'score': 0.7856793999671936, 'sentence': 'Seferler Haziran ayında başlıyor.'}]
[{'label': 'negative', 'score': 0.8989760875701904, 'sentence': 'Siyasette temiz insanlara ihtiyacımız var.'}]
[{'label': 'positive', 'score': 0.994697093963623, 'sentence': 'Broz, büyükbabasının başarılarını gururla anıyor.'}]
[{'label': 'negative', 'score': 0.8941656947135925, 'sentence': 'Calasan yetenekli çocuk bursu da aldı.'}]
[{'label': 'positive', 'score': 0.6083768010139465, 'sentence': 'Referandum tarihi henüz belirlenmedi.'}]
[{'label': 'positive', 'score': 0.9253294467926025, 'sentence': 'Her yaştan takasçı geliyor.'}]
[{'label': 'positive', 'score': 0.6

In [5]:
excelData['Sentiment'].value_counts() #2956 negatif ve 2045 pozitif veri elde edilmiştir.

negative    2956
positive    2045
Name: Sentiment, dtype: int64